In [2]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import re, os
from pathlib import Path
from glob import glob
import matplotlib.pyplot as plt
from statistics import mode

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import Ridge
from sklearn.metrics import log_loss

def Read():
  train = pd.read_csv(path_start/"input/df_train.csv")
  test = pd.read_csv(path_start/"input/test.csv")
  df_ConCat = pd.concat([train, test])
  df_ConCat = df_ConCat.drop(["Unnamed: 0", "地域", "前面道路：方位", "前面道路：種類",	"前面道路：幅員（ｍ）", 
                              "種類", "土地の形状", "間口", "延床面積（㎡）"], axis=1)
  return df_ConCat

path_start = Path("/content/drive/MyDrive/Nishika/Mansion_2023_spring")
path_import = path_start/"Inport_data/PLOT_utf8"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
df = pd.DataFrame()
for i in glob(str(path_import/"*.csv")):
  df = pd.concat([df, pd.read_csv(i, encoding="utf-8")], axis=0)
display(df.shape)
display(df.head(3))

(190388, 10)

,都道府県コード,都道府県名,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,緯度,経度,原典資料コード,大字・字・丁目区分コード
0,1,北海道,1101,札幌市中央区,1.101000e+10,旭ケ丘一丁目,43.042230,141.319722,0,3
1,1,北海道,1101,札幌市中央区,1.101000e+10,旭ケ丘二丁目,43.039768,141.321733,0,3
2,1,北海道,1101,札幌市中央区,1.101000e+10,旭ケ丘三丁目,43.039569,141.319617,0,3


In [27]:
a = df.groupby("市区町村コード").mean().reset_index()
a.head()

<ipython-input-27-8e7f690ad1d3>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  a = df.groupby("市区町村コード").mean().reset_index()


,市区町村コード,都道府県コード,大字町丁目コード,緯度,経度,原典資料コード,大字・字・丁目区分コード
0,1101,1.0,1.101004e+10,43.053101,141.337146,0.0,2.987879
1,1102,1.0,1.102006e+10,43.114164,141.338905,0.0,2.979186
2,1103,1.0,1.103004e+10,43.096829,141.373923,0.0,2.987330
3,1104,1.0,1.104008e+10,43.052676,141.414699,0.0,2.352174
4,1105,1.0,1.105003e+10,43.030463,141.383493,0.0,2.988743


In [32]:
df_a = a[["市区町村コード", "緯度", "経度"]]
df_a.head()

,市区町村コード,緯度,経度
0,1101,43.053101,141.337146
1,1102,43.114164,141.338905
2,1103,43.096829,141.373923
3,1104,43.052676,141.414699
4,1105,43.030463,141.383493


In [31]:
train = pd.read_csv(path_start/"input/df_train.csv", index_col=0)
test = pd.read_csv(path_start/"input/test.csv", index_col=0)
df_ConCat = pd.concat([train, test])
df_ConCat = df_ConCat[["ID", "市区町村コード"]]
df_ConCat.head()

,ID,市区町村コード
0,8067540,8217
1,8027265,8219
2,8061526,8220
3,8086147,8220
4,8049498,8224


In [34]:
b = pd.merge(df_ConCat, df_a, on="市区町村コード")
df_b = b[["ID", "緯度", "経度"]]
df_b.head()

,ID,緯度,経度
0,8067540,35.911886,140.072573
1,8072555,35.911886,140.072573
2,8024138,35.911886,140.072573
3,8067758,35.911886,140.072573
4,8045321,35.911886,140.072573


In [38]:
aim = ["緯度", "経度"]
for i in aim:
  df_b[i] = (df_b[i] - df_b[i].mean())/ df_b[i].std()
df_b.head()

<ipython-input-38-ad986a36a023>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_b[i] = (df_b[i] - df_b[i].mean())/ df_b[i].std()


,ID,緯度,経度
0,8067540,0.206981,0.758921
1,8072555,0.206981,0.758921
2,8024138,0.206981,0.758921
3,8067758,0.206981,0.758921
4,8045321,0.206981,0.758921


In [39]:
df_b.to_csv(path_start/"feature_value/df_緯度経度.csv", index=False)
pd.read_csv(path_start/"feature_value/df_緯度経度.csv")

,ID,緯度,経度
0,8067540,0.206981,0.758921
1,8072555,0.206981,0.758921
2,8024138,0.206981,0.758921
3,8067758,0.206981,0.758921
4,8045321,0.206981,0.758921
...,...,...,...
784708,28194896,-0.705696,-1.079587
784709,28194894,-0.705696,-1.079587
784710,28194897,-0.705696,-1.079587
784711,28195119,-0.705696,-1.079587
